In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model, metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
import datetime
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import torch
from torch import nn

# def shifting_ace(df, values_for_shift):
#     new_df1 = df.iloc[:, 0:9]
#     new_columns = {}
#     for key, value in values_for_shift.items():
#         new_columns[key] = df[key]
#         for i in range(1, value+1):
#             new_columns[f'{key} -{i}'] = df[key].shift(periods=i, fill_value=np.nan)
#     new_df2 = pd.DataFrame(new_columns)
#     new_df = pd.concat([new_df1, new_df2], axis=1)
#     return new_df

def shifting_discover(df, values_for_shift):
    new_df1 = df.iloc[:, 0:4]
    new_columns = {}
    for key, value in values_for_shift.items():
        new_columns[key] = df[key]
        for i in range(1, value+1):
            new_columns[f'{key} -{i}'] = df[key].shift(periods=i, fill_value=np.nan)
    new_df2 = pd.DataFrame(new_columns)
    new_df = pd.concat([new_df1, new_df2], axis=1)
    return new_df

def shifting_ace(df, values_for_shift, future_shift):
    new_df1 = df.iloc[:, 0:9]
    new_columns = {}
    for key, value in values_for_shift.items():
        if key == 'Dst':
            for i in range(0, future_shift+value+1):
                if i < future_shift:
                    new_columns[f'{key} +{future_shift-i}'] = df[key].shift(periods=i, fill_value=np.nan)
                elif i == future_shift:
                    new_columns[key] = df[key].shift(periods=i, fill_value=np.nan)
                else:
                    new_columns[f'{key} -{i - future_shift}'] = df[key].shift(periods=i, fill_value=np.nan)
        else: 
            new_columns[key] = df[key]
            for i in range(1, value+1):
                new_columns[f'{key} -{i}'] = df[key].shift(periods=i, fill_value=np.nan)
    new_df2 = pd.DataFrame(new_columns)
    new_df = pd.concat([new_df1, new_df2], axis=1)
    return new_df

In [12]:
df = pd.read_csv(r"Compare_ACE_DSCOVR.csv", sep=';', na_values='N', decimal=',')

values = ['Dst', 'B_x', 'B_gsm_y', 'B_gsm_z', 'B_magn', 'H_den_SWP', 'SW_spd']
nums_of_shifts = [24, ]*7
future_shifting = 1 #горизонт предсказания
#nums_of_shifts = [43, 26, 12, 3, 19, 56, 16]

names = list(df.iloc[:, 9:])
scaler = StandardScaler() #нормировка
df.iloc[:, 9:] = scaler.fit_transform(df.iloc[:, 9:])
df.iloc[:, 9:] = pd.DataFrame(df.iloc[:, 9:], columns=names)

values_for_shift = dict(zip(values, nums_of_shifts))
df = shifting_ace(df, values_for_shift, future_shifting) #погружение

# пример как осуществлено погружение
check_df = df.iloc[:10, 7:25]
check_df

C:\Users\Пятка\AppData\Local\Temp\ipykernel_21860\2808674637.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.11013505 -0.16038264 -0.16038264 ...  0.04060773  0.09085532
  0.04060773]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 9:] = scaler.fit_transform(df.iloc[:, 9:])


,doyCos,hourCos,Dst +1,Dst,Dst -1,Dst -2,Dst -3,Dst -4,Dst -5,Dst -6,Dst -7,Dst -8,Dst -9,Dst -10,Dst -11,Dst -12,Dst -13,Dst -14
0,-0.862961,-0.130526,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.862598,-0.382683,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.862235,-0.608761,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.861872,-0.793353,-0.059887,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.861508,-0.923880,-0.059887,-0.059887,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.861143,-0.991445,-0.110135,-0.059887,-0.059887,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-0.860779,-0.991445,-0.160383,-0.110135,-0.059887,-0.059887,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.860413,-0.923880,-0.160383,-0.160383,-0.110135,-0.059887,-0.059887,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-0.860048,-0.793353,-0.160383,-0.160383,-0.160383,-0.110135,-0.059887,-0.059887,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-0.859681,-0.608761,-0.059887,-0.160383,-0.160383,-0.160383,-0.110135,-0.059887,-0.059887,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#добавляю дату-время на место индексов

new_values = list(df.iloc[:, 9:].columns)

date_values = ['year', 'month', 'day', 'hour from']
date = pd.DataFrame(df.loc[:, date_values])

date['hour from'] = date['hour from'].map(lambda x: '0'+str(x) if x not in range(10, 24) else str(x))
date['datetime'] = date['year'].astype(str) + date['month'].astype(str) + date['day'].astype(str) + ' ' + date['hour from']
date['datetime'] = pd.to_datetime(date['datetime'])

dataset = df[new_values]
dataset = dataset.set_index(date['datetime'])

dataset_ACE = dataset #отсюда взять значения дст и соединить с данными с discover
dataset.iloc[:10]

,Dst +1,Dst,Dst -1,Dst -2,Dst -3,Dst -4,Dst -5,Dst -6,Dst -7,Dst -8,...,SW_spd -15,SW_spd -16,SW_spd -17,SW_spd -18,SW_spd -19,SW_spd -20,SW_spd -21,SW_spd -22,SW_spd -23,SW_spd -24
datetime,,,,,,,,,,,,,,,,,,,,,
1997-10-21 00:00:00,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-21 01:00:00,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-21 02:00:00,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-21 03:00:00,-0.059887,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-21 04:00:00,-0.059887,-0.059887,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-21 05:00:00,-0.110135,-0.059887,-0.059887,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-21 06:00:00,-0.160383,-0.110135,-0.059887,-0.059887,-0.160383,-0.160383,-0.110135,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-21 07:00:00,-0.160383,-0.160383,-0.110135,-0.059887,-0.059887,-0.160383,-0.160383,-0.110135,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-21 08:00:00,-0.160383,-0.160383,-0.160383,-0.110135,-0.059887,-0.059887,-0.160383,-0.160383,-0.110135,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
dataset = dataset.dropna()

dataset

,Dst +1,Dst,Dst -1,Dst -2,Dst -3,Dst -4,Dst -5,Dst -6,Dst -7,Dst -8,...,SW_spd -15,SW_spd -16,SW_spd -17,SW_spd -18,SW_spd -19,SW_spd -20,SW_spd -21,SW_spd -22,SW_spd -23,SW_spd -24
datetime,,,,,,,,,,,,,,,,,,,,,
1997-10-22 01:00:00,0.693826,0.693826,0.744074,0.844569,0.844569,0.492836,0.442588,0.442588,0.392341,0.342093,...,-1.192274,-1.183867,-1.161917,-1.170082,-1.154669,-1.151457,-1.157589,-1.123825,-1.096198,-1.143193
1997-10-22 02:00:00,0.392341,0.693826,0.693826,0.744074,0.844569,0.844569,0.492836,0.442588,0.442588,0.392341,...,-1.198088,-1.192274,-1.183867,-1.161917,-1.170082,-1.154669,-1.151457,-1.157589,-1.123825,-1.096198
1997-10-22 03:00:00,0.241598,0.392341,0.693826,0.693826,0.744074,0.844569,0.844569,0.492836,0.442588,0.442588,...,-1.255216,-1.198088,-1.192274,-1.183867,-1.161917,-1.170082,-1.154669,-1.151457,-1.157589,-1.123825
1997-10-22 04:00:00,0.442588,0.241598,0.392341,0.693826,0.693826,0.744074,0.844569,0.844569,0.492836,0.442588,...,-1.286637,-1.255216,-1.198088,-1.192274,-1.183867,-1.161917,-1.170082,-1.154669,-1.151457,-1.157589
1997-10-22 05:00:00,0.543084,0.442588,0.241598,0.392341,0.693826,0.693826,0.744074,0.844569,0.844569,0.492836,...,-1.242391,-1.286637,-1.255216,-1.198088,-1.192274,-1.183867,-1.161917,-1.170082,-1.154669,-1.151457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-24 19:00:00,0.945064,0.693826,0.693826,0.492836,0.492836,0.291846,-0.009640,-0.160383,-0.260878,-0.210630,...,-0.009486,0.084588,0.139337,0.169209,0.213778,0.238898,0.293285,0.324841,0.360473,0.386368
2021-03-24 20:00:00,0.894817,0.945064,0.693826,0.693826,0.492836,0.492836,0.291846,-0.009640,-0.160383,-0.260878,...,0.022459,-0.009486,0.084588,0.139337,0.169209,0.213778,0.238898,0.293285,0.324841,0.360473
2021-03-24 21:00:00,0.241598,0.894817,0.945064,0.693826,0.693826,0.492836,0.492836,0.291846,-0.009640,-0.160383,...,-0.041562,0.022459,-0.009486,0.084588,0.139337,0.169209,0.213778,0.238898,0.293285,0.324841


In [5]:
data_discover = pd.read_csv(r'Discover_с_интреполяцией12_актуальные_данные_до_22.09.2024.csv', sep=';', decimal=',')

names = ['bx_gsm', 'by_gsm', 'bz_gsm', 'bt', 'proton_density', 'proton_speed', 'proton_temperature']
nums_of_shifts = [24, ]*7

# Dst - 43, Bx - 26, B_GSM_y - 12, B_GSM_z - 3, B_magn - 19, SW_spd - 56, H_den - 16

# обрезка данных до даты окончания данных с ACE
## 40715 = 25.03.2021 11:00

data_discover = data_discover.iloc[:40716]
#data_discover

In [6]:
values_for_shift_discover = dict(zip(names, nums_of_shifts))
data_discover = shifting_discover(data_discover, values_for_shift_discover)

# names = list(data_discover)

# scaler = StandardScaler()
# data_discover = scaler.fit_transform(data_discover)
# data_discover_scaled = pd.DataFrame(data_discover, columns=names)
# #data_discover_scaled = data_discover_scaled.set_index(date['datetime'])

# data_discover_scaled
# data_discover = data_dicover_scaled.dropna()

## Значения дст с 27.07.2016 0:00 по 25.03.2021 11:00
# dst_ACE = dataset_ACE.iloc[164496:, 0:44]

# dst_ACE = dataset_ACE.iloc[164496:, 0:1]
# dst_ACE

#data_discover = data_discover + dst_ACE
#print(len(data_discover), len(dst_ACE))

In [19]:
if not dataset.index.is_monotonic_increasing:
   dataset = dataset.sort_index()

#последняя дата - 2021-12-09 23:00:00

# x_train = dataset.loc[:'2018-01-01 01:00:00', dataset.columns[future_shifting:]]
# x_test = dataset.loc['2018-01-01 02:00:00':, dataset.columns[future_shifting:]]
# y_train = dataset['Dst +1'].loc[:'2018-01-01 01:00:00']
# y_test = dataset['Dst +1'].loc['2018-01-01 02:00:00':]

x_train = dataset.loc[:'2020-10-17 01:00:00', dataset.columns[future_shifting:]]
x_test = dataset.loc['2020-10-17 02:00:00':, dataset.columns[future_shifting:]]
y_train = dataset['Dst +1'].loc[:'2020-10-17 01:00:00']
y_test = dataset['Dst +1'].loc['2020-10-17 02:00:00':]
 
x_train

,Dst,Dst -1,Dst -2,Dst -3,Dst -4,Dst -5,Dst -6,Dst -7,Dst -8,Dst -9,...,SW_spd -15,SW_spd -16,SW_spd -17,SW_spd -18,SW_spd -19,SW_spd -20,SW_spd -21,SW_spd -22,SW_spd -23,SW_spd -24
datetime,,,,,,,,,,,,,,,,,,,,,
1997-10-22 01:00:00,0.693826,0.744074,0.844569,0.844569,0.492836,0.442588,0.442588,0.392341,0.342093,0.241598,...,-1.192274,-1.183867,-1.161917,-1.170082,-1.154669,-1.151457,-1.157589,-1.123825,-1.096198,-1.143193
1997-10-22 02:00:00,0.693826,0.693826,0.744074,0.844569,0.844569,0.492836,0.442588,0.442588,0.392341,0.342093,...,-1.198088,-1.192274,-1.183867,-1.161917,-1.170082,-1.154669,-1.151457,-1.157589,-1.123825,-1.096198
1997-10-22 03:00:00,0.392341,0.693826,0.693826,0.744074,0.844569,0.844569,0.492836,0.442588,0.442588,0.392341,...,-1.255216,-1.198088,-1.192274,-1.183867,-1.161917,-1.170082,-1.154669,-1.151457,-1.157589,-1.123825
1997-10-22 04:00:00,0.241598,0.392341,0.693826,0.693826,0.744074,0.844569,0.844569,0.492836,0.442588,0.442588,...,-1.286637,-1.255216,-1.198088,-1.192274,-1.183867,-1.161917,-1.170082,-1.154669,-1.151457,-1.157589
1997-10-22 05:00:00,0.442588,0.241598,0.392341,0.693826,0.693826,0.744074,0.844569,0.844569,0.492836,0.442588,...,-1.242391,-1.286637,-1.255216,-1.198088,-1.192274,-1.183867,-1.161917,-1.170082,-1.154669,-1.151457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-16 21:00:00,0.442588,0.593331,0.693826,1.447540,0.744074,0.744074,0.744074,0.844569,0.894817,0.844569,...,-1.325813,-1.315305,-1.294577,-1.311979,-1.330926,-1.350886,-1.317258,-1.359772,-1.390581,-1.357095
2020-10-16 22:00:00,0.392341,0.442588,0.593331,0.693826,1.447540,0.744074,0.744074,0.744074,0.844569,0.894817,...,-1.309728,-1.325813,-1.315305,-1.294577,-1.311979,-1.330926,-1.350886,-1.317258,-1.359772,-1.390581
2020-10-16 23:00:00,0.392341,0.392341,0.442588,0.593331,0.693826,1.447540,0.744074,0.744074,0.744074,0.844569,...,-1.283132,-1.309728,-1.325813,-1.315305,-1.294577,-1.311979,-1.330926,-1.350886,-1.317258,-1.359772


In [7]:
len(x_train), len(x_test), len(y_train), len(y_test)

(175223, 28230, 175223, 28230)

In [8]:
x_test

,Dst,Dst -1,Dst -2,Dst -3,Dst -4,Dst -5,Dst -6,Dst -7,Dst -8,Dst -9,...,SW_spd -15,SW_spd -16,SW_spd -17,SW_spd -18,SW_spd -19,SW_spd -20,SW_spd -21,SW_spd -22,SW_spd -23,SW_spd -24
datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-01 02:00:00,0.241598,0.342093,0.492836,0.593331,0.995312,1.347045,1.497788,1.246550,0.995312,0.945064,...,-0.730954,-0.675607,-0.586112,-0.548328,-0.583438,-0.698411,-0.770547,-0.746006,-0.734602,-0.737045
2018-01-01 03:00:00,-0.059887,0.241598,0.342093,0.492836,0.593331,0.995312,1.347045,1.497788,1.246550,0.995312,...,-0.729714,-0.730954,-0.675607,-0.586112,-0.548328,-0.583438,-0.698411,-0.770547,-0.746006,-0.734602
2018-01-01 04:00:00,0.141103,-0.059887,0.241598,0.342093,0.492836,0.593331,0.995312,1.347045,1.497788,1.246550,...,-0.726256,-0.729714,-0.730954,-0.675607,-0.586112,-0.548328,-0.583438,-0.698411,-0.770547,-0.746006
2018-01-01 05:00:00,-0.009640,0.141103,-0.059887,0.241598,0.342093,0.492836,0.593331,0.995312,1.347045,1.497788,...,-0.717265,-0.726256,-0.729714,-0.730954,-0.675607,-0.586112,-0.548328,-0.583438,-0.698411,-0.770547
2018-01-01 06:00:00,-0.311125,-0.009640,0.141103,-0.059887,0.241598,0.342093,0.492836,0.593331,0.995312,1.347045,...,-0.723301,-0.717265,-0.726256,-0.729714,-0.730954,-0.675607,-0.586112,-0.548328,-0.583438,-0.698411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-09 19:00:00,-0.009640,-0.009640,-0.009640,0.040608,-0.009640,-0.160383,-0.210630,-0.210630,-0.110135,-0.009640,...,-0.415171,-0.327362,-0.318662,-0.313790,-0.281579,-0.389925,-0.407692,-0.369381,-0.267501,-0.257261
2021-12-09 20:00:00,0.040608,-0.009640,-0.009640,-0.009640,0.040608,-0.009640,-0.160383,-0.210630,-0.210630,-0.110135,...,-0.374209,-0.415171,-0.327362,-0.318662,-0.313790,-0.281579,-0.389925,-0.407692,-0.369381,-0.267501
2021-12-09 21:00:00,0.141103,0.040608,-0.009640,-0.009640,-0.009640,0.040608,-0.009640,-0.160383,-0.210630,-0.210630,...,-0.395008,-0.374209,-0.415171,-0.327362,-0.318662,-0.313790,-0.281579,-0.389925,-0.407692,-0.369381


In [9]:
y_train

datetime
1997-10-22 01:00:00    0.693826
1997-10-22 02:00:00    0.392341
1997-10-22 03:00:00    0.241598
1997-10-22 04:00:00    0.442588
1997-10-22 05:00:00    0.543084
                         ...   
2017-12-31 21:00:00    0.995312
2017-12-31 22:00:00    0.593331
2017-12-31 23:00:00    0.492836
2018-01-01 00:00:00    0.342093
2018-01-01 01:00:00    0.241598
Name: Dst +1, Length: 175223, dtype: float64

In [10]:
y_test

datetime
2018-01-01 02:00:00   -0.059887
2018-01-01 03:00:00    0.141103
2018-01-01 04:00:00   -0.009640
2018-01-01 05:00:00   -0.311125
2018-01-01 06:00:00   -0.411621
                         ...   
2021-12-09 19:00:00    0.040608
2021-12-09 20:00:00    0.141103
2021-12-09 21:00:00    0.191351
2021-12-09 22:00:00    0.090855
2021-12-09 23:00:00    0.090855
Name: Dst +1, Length: 28230, dtype: float64

In [18]:
# pipe = make_pipeline(MinMaxScaler(), LinearRegression())
# pipe.fit(x_train, y_train)
# y_pred = pipe.predict(x_test)

model = linear_model.LinearRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

R_2 = metrics.r2_score(y_test, y_pred)
MSE = metrics.mean_squared_error(y_test, y_pred)
MAE = metrics.mean_absolute_error(y_test, y_pred)

print('R**2 score:', R_2)
print('MSE:', MSE)
print('MAE:', MAE)

R**2 score: 0.924577729880904
MSE: 0.021078046756107503
MAE: 0.09826257858592387


In [25]:
# plt.hist(MSE, color = 'blue', edgecolor = 'black',
#          bins = 5)

# plt.xlabel('ACE')
# plt.ylabel('MSE, нТл')

In [ ]:
# corr_matrix = dataset.corr()

# plt.figure(figsize=(6,5))
# sns.heatmap(corr_matrix)
# plt.show()

In [ ]:
plt.plot(y_test, c='blue')
plt.plot(y_pred, c='red')
plt.show()

In [ ]:
import statsmodels.api as sm

#res = sm.tsa.seasonal_decompose(dataset['Dst'])
# plt.rc("figure", figsize=(25,8))
# resplot = res.plot()
res = dataset['Dst']
res